In [303]:
import os
import math
import time

import numpy as np

In [294]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __str__(self):
        return "{} {}".format(str(self.p1), str(self.p2)) 
    
    def __eq__(self, other):
        if self.p1 == other.p1 or self.p1 == other.p2:
            if self.p2 == other.p1 or self.p2 == other.p2:
                return True
        return False
    
    
class Face:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        self.e1 = Edge(p1, p2)
        self.e2 = Edge(p2, p3)
        self.e3 = Edge(p3, p1)
        
    def __str__(self):
        return "{} {} {}".format(str(self.p1), str(self.p2), str(self.p3))
    
    def __eq__(self, other):
        pass

In [279]:
def cross(a, b):
    return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

def dot(a, b):
    return (a.x*b.x)+(a.y*b.y)+(a.z*b.z)

def norma(a):
    return math.sqrt(a.x*a.x + a.y*a.y + a.z*a.z)

def angle(a, b):
    normaa = norma(a)
    normab = norma(b)
    if norma(a)==0 or norma(b) == 0:
        print("ANGULO DIV ZERO")
        return -999999999
    return dot(a, b) / (norma(a)*norma(b))

def sub(a, b):
    return Point(a.x-b.x, a.y-b.y, a.z-b.z)

def normal(face):
    v1 = sub(face.p2, face.p1)    
    v2 = sub(face.p3, face.p1)
    return cross(v1,v2)

In [280]:
def find_min(Points):
    ys = [p.y for p in Points]
    min_index = ys.index(min(ys))
    return Points[min_index]

In [281]:
def check_face(F, f):
    for face in F:
        if face == f:
            return False
    return True

In [282]:
def check_edge_face(F, e):
    count = 0
    
    for face in F:
        if face.e1 == e or face.e2 == e or face.e3 == e:
            count+=1
    if count > 1:
        return False
    return True

In [283]:
def check_edges(E, e):
    for edge in E:
        if edge == e:
            return False
    return True

In [284]:
def free_edge(f):
    return [f.e3, f.e1, f.e2]

In [285]:
def find_first_face(P):
    p1 = find_min(P)
    p2 = Point(p1.x+1, p1.y, p1.z)    # p_linha
    p3 = Point(p1.x, p1.y, p1.z-1)    # p2_linha
    
    ftemp = Face(p1,p3,p2)
    normal1 = normal(ftemp)
    
    max_angle = -999999
    for point in P:
        if point == ftemp.p1: continue
        f_ = Face(point, ftemp.p3, ftemp.p1)
        normal2 = normal(f_)
        ang = angle(normal1, normal2)
        if ang > max_angle:
            p3 = point
            max_angle = ang
    
    ftemp = Face(p1,p3,p2)
    normal1 = normal(ftemp)
    max_angle = -999999
    for point in P:
        if point == ftemp.p1 or point == ftemp.p2: continue
        f_ = Face(point, ftemp.p2, ftemp.p1)
        normal2 = normal(f_)
        ang = angle(normal1, normal2)
        if ang > max_angle:
            p2 = point
            max_angle = ang
    return Face(p2,p3,p1)    

In [286]:
from time import sleep

In [288]:
def gift_wrapping(P):
    f = find_first_face(P)
    hull = [f]
    faces_iter = [f]
    free_edges = []
    face_atual = None
    candidate = None
    while len(faces_iter) > 0:
        face_atual = faces_iter.pop(0)
        free_edges = free_edge(face_atual)
        
        normal1 = normal(face_atual)
        for e in free_edges:
            max_angle = -99999
            candidate = None
            if check_edge_face(hull, e):
                for p in P:
                    if p == e.p1 or p == e.p2:
                        continue
                    f_ = Face(e.p1, p, e.p2)
                    normal2 = normal(f_)
                    ang = angle(normal1,normal2)
                    if ang > max_angle:
                        if check_face(hull, f_):
                            max_angle = ang
                            candidate = p
                face_no_hull = Face(e.p1, candidate, e.p2)
                hull.append(face_no_hull)
                faces_iter.append(face_no_hull)
    return hull

In [289]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            try:
                points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                return points
    return points

In [290]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                i1 = j
            if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                i2 = j
            if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                i3 = j
                    
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [312]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                    i1 = j
                if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                    i2 = j
                if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [304]:
pth = "/Users/romul/Documents/UFC/MESTRADO/Disciplinas/Geometria Computacional/obj/"

In [305]:
cilindro = read_vertices_from_obj(pth+"Debug.obj")
# for c in cilindro: print(c)
ch = gift_wrapping(cilindro)
print(len(ch))
write_faces(ch, cilindro, os.getcwd()+"\\obj\\hullCilindro.obj")
# print_faces(ch, cilindro)

28


In [250]:
points = []
for i in range(1000):
    a = np.random.random(1)
    b = np.random.random(1)
    c = np.random.random(1)
    
    points.append(Point(a[0],b[0],c[0]))

print(len(points))


1000


In [306]:
H = gift_wrapping(points)
print(len(H))
write_faces(H, points, os.getcwd()+"\\obj\\hull1000pontos.obj")


122


In [307]:
esfera = read_vertices_from_obj(pth+"esfera.obj")
# for c in cilindro: print(c)
ch = gift_wrapping(esfera)
print(len(ch))
write_faces(ch, esfera, os.getcwd()+"\\obj\\hullEsfera.obj")

960


In [308]:
bento = read_vertices_from_obj(pth+"bento.obj")
# for c in cilindro: print(c)
ch = gift_wrapping(bento)
print(len(ch))
write_faces(ch, bento, os.getcwd()+"\\obj\\hullBento.obj")

8


In [311]:
controle = read_vertices_from_obj(os.getcwd()+"\\obj\\controle.obj")
print(len(controle))
hullControle = gift_wrapping(controle)
print(len(hullControle))
write_faces(hullControle, controle, os.getcwd()+"\\obj\\hullControle.obj")

348
433


In [269]:
nintendo = read_vertices_from_obj(os.getcwd()+"\\obj\\super-nintendo.obj")
print(len(nintendo))

60298


In [274]:
t1 = time.time()
hullNintendo = gift_wrapping(nintendo)
print(time.time()-t1)
print(len(hullNintendo))
write_faces(hullNintendo, nintendo, os.getcwd()+"\\obj\\hullNintendo.obj")

ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO DIV ZERO
ANGULO D